In [ ]:
#week 2(10-17th, 2021): Creating DMC(Duff moisture content) mode


import math
import datetime
import numpy as np
import pandas as pd

def daylength(dayOfYear, lat):
    """Computes the length of the day (the time between sunrise and
    sunset) given the day of the year and latitude of the location.
    Function uses the Brock model for the computations.
    For more information see, for example,
    Forsythe et al., "A model comparison for daylength as a
    function of latitude and day of year", Ecological Modelling,
    1995.
    Parameters
    ----------
    dayOfYear : int
        The day of the year. 1 corresponds to 1st of January
        and 365 to 31st December (on a non-leap year).
    lat : float
        Latitude of the location in degrees. Positive values
        for north and negative for south.
    Returns
    -------
    d : float
        Daylength in hours.
    """
    latInRad = np.deg2rad(lat)
    declinationOfEarth = 23.45*np.sin(np.deg2rad(360.0*(283.0+dayOfYear)/365.0))
    if -np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth)) <= -1.0:
        return 24.0
    elif -np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth)) >= 1.0:
        return 0.0
    else:
        hourAngle = np.rad2deg(np.arccos(-np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth))))
        return 2.0*hourAngle/15.0

#DMC Model

def DMC(TEMP,RH,RAIN,DMCPrev,dayOfYear, lat):
    '''Calculates today's Duff Moisture Code
Parameters:
    TEMP is the 12:00 LST temperature in degrees celsius
    RH is the 12:00 LST relative humidity in %
    RAIN is the 24-hour accumulated rainfall in mm, calculated at 12:00 LST
    DMCstart is the start dmc
    DMCPrev is the prevvious day's DMC
    lat is the latitude in decimal degrees of the location for which calculations are being made
    dayOfYear is the day of the year(int).

    DMC(17,42,0,6,94,45.98)'''
    RH = min(100.0,RH)
    if RAIN > 1.5:
        re = 0.92 * RAIN - 1.27

        mo = 20.0 + math.exp(5.6348 - DMCPrev / 43.43)

        if DMCPrev <= 33.0:
            b = 100.0 / (0.35 + 0.3 * DMCPrev)
        else:
            if DMCPrev <= 65.0:
                b = 14.0 - 1.3 * math.log(DMCPrev)
            else:
                b = 6.2 * math.log(DMCPrev) - 17.2
        
        mr = mo + 1000.0 * re / (48.77 + b * re)

        pr = 244.72 - 43.43 * math.log(mr - 20.0)

        if pr > 0.0:
            DMCPrev = pr
        else:
            DMCPrev = 0.0

    if TEMP > -1.1:
        d1 = daylength(dayOfYear, lat)

        k = 1.894 * (TEMP + 1.1) * (100.0 - RH) * d1 * 0.000001

    else:
        k = 0.0

    return DMCPrev + 100.0 * k

#data combining of stie data(TEMP,RH) + atmosphere data(Rain)
#site weather station( North Fraser )
fields =["TIMESTAMP","AirTC","RH"]
df=pd.read_csv("NF_ws.csv",skipinitialspace=True,usecols=fields)
time_need=["12:00"]
b = [any(i.lower() in j.lower() for i in time_need) for j in df["TIMESTAMP"].values]
df=df[b]
date=df["TIMESTAMP"]

#add day of year 
date = pd.to_datetime(date, format='%Y%m%d %H:%M:%S',errors='coerce')
dayOfYear =date.dt.dayofyear

df['dayOfYear'] = dayOfYear

df.reset_index(drop=True, inplace=True)
#PG airport weather station
fields1 =["Date/Time","Total Precip (mm)"]
df1=pd.read_csv("pgap.csv",skipinitialspace=True,usecols=fields1)
df1.fillna(0, inplace=True)

date1=pd.to_datetime(df1["Date/Time"])
a=(date1>='2021-06-15')&(date1<='2021-09-30')
newdf1=df1.loc[a]
newdf1.reset_index(drop=True, inplace=True)
print(newdf1)

#combine them together 
df["Pg_ar_Rain"]=newdf1["Total Precip (mm)"]

    
#weather condition> DMC
DMCPrev=10 #assume numebr as a start 
count=0#data frame length 
Temp=df["AirTC"]
RH=df["RH"]
RAIN=df["Pg_ar_Rain"]
dayOfYear=df["dayOfYear"]
DMC_List=[]
while count<108:
    DMCPrev=DMC(Temp[count],RH[count],RAIN[count],DMCPrev,dayOfYear[count], 54)
    DMC_List.append(DMCPrev)
    count+=1
    
# plot 
import matplotlib.pyplot as plt
plt.plot(DMC_List)


In [ ]:
#week 3(17th-24th)
FFMC model Build up
#FFMC model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
def FFMC(TEMP,RH,WIND,RAIN,FFMCPrev):
    '''Calculates today's Fine Fuel Moisture Code
Parameters:
    TEMP is the 12:00 LST temperature in degrees celsius
    RH is the 12:00 LST relative humidity in %
    RAIN is the 24-hour accumulated rainfall in mm, calculated at 12:00 LST
    FFMCPrev is the previous day's FFMC

    FFMC(17,42,25,0,85) = 87.692980092774448'''

    RH = min(100.0,RH)
    mo = 147.2 * (101.0 - FFMCPrev) / (59.5 + FFMCPrev)

    if RAIN > .5:
        rf = RAIN - .5

        if mo <= 150.0:
            mr = mo + \
                 42.5 * rf * math.exp(-100.0 / (251.0 - mo)) * (1.0-math.exp(-6.93 / rf))
        else:

            mr = mo + \
                 42.5 * rf * math.exp(-100.0 / (251.0 - mo)) * (1.0-math.exp(-6.93 / rf)) + \
                 0.0015 * pow(mo - 150.0, 2) * pow(rf, .5)

        if mr > 250.0:
            mr = 250.0

        mo=mr

    ed = 0.942 * pow(RH, 0.679) + \
         11.0 * math.exp((RH - 100.0) / 10.0) + 0.18 * (21.1 - TEMP) * (1.0 - math.exp(-0.115 * RH))

    if mo > ed:
        ko = 0.424 * (1.0 - pow(RH / 100.0, 1.7)) + \
             0.0694 * pow(WIND, .5) * (1.0 - pow(RH / 100.0, 8))

        kd = ko * 0.581 * math.exp(0.0365 * TEMP)

        m = ed + (mo - ed) * pow(10.0,-kd)

    else:
        ew = 0.618 * pow(RH,0.753) + \
             10.0 * math.exp((RH - 100.0) / 10.0) + \
             0.18 * (21.1 - TEMP) * (1.0 - math.exp(-0.115 * RH))
        if mo < ew:
            k1 = 0.424 * (1.0 - pow((100.0 - RH) / 100.0, 1.7)) + \
                 0.0694 * pow(WIND, .5) * (1.0 - pow((100.0 - RH) / 100.0, 8))

            kw = k1 * 0.581 * math.exp(0.0365 * TEMP)

            m = ew - (ew - mo) * pow(10.0, -kw)
        else:
            m = mo
    return 59.5 * (250.0 - m) / (147.2 + m)

#data combining of stie data(TEMP,RH) + atmosphere data(Rain)
#site weather station( North Fraser )
fields =["TIMESTAMP","AirTC","RH","Wind_Speed_m_s"]
df=pd.read_csv("NF_ws.csv",skipinitialspace=True,usecols=fields)
time_need=["12:00"]
b = [any(i.lower() in j.lower() for i in time_need) for j in df["TIMESTAMP"].values]
df=df[b]
date=df["TIMESTAMP"]

df.reset_index(drop=True, inplace=True)
#PG airport weather station
fields1 =["Date/Time","Total Precip (mm)"]
df1=pd.read_csv("pgap.csv",skipinitialspace=True,usecols=fields1)
df1.fillna(0, inplace=True)

date1=pd.to_datetime(df1["Date/Time"])
a=(date1>='2021-06-15')&(date1<='2021-09-30')
newdf1=df1.loc[a]
newdf1.reset_index(drop=True, inplace=True)
print(newdf1)

#combine them together 
df["Pg_ar_Rain"]=newdf1["Total Precip (mm)"]



#re-number the index

print(df)
#combine site +atomphsere 


#weather condition> FFMC
FFMCPrev=50 #assume numebr as a start 
count=0#data frame length 
Temp=df["AirTC"]
RH=df["RH"]
RAIN=df["Pg_ar_Rain"]
Wind=df["Wind_Speed_m_s"]
FFMC_List=[]
while count<108:
    FFMCPrev=FFMC(Temp[count],RH[count],Wind[count]/3.6,RAIN[count],FFMCPrev)
    FFMC_List.append(147.2*(101-FFMCPrev)/(59.2+FFMCPrev))
    count+=1

plt.plot(FFMC_List)

In [ ]:
#week 4(24th-31st)
#combining of field data+ emperical data 
df["FMC"]=FFMC_List
fields2=["Location","Stand_Type","Fuel_Type","Date_Collected","Moisture_Percent"]
df2=pd.read_csv("2021PgFuel.csv",skipinitialspace=True,usecols=fields2)
location=df2["Location"]=="NFraser"
fuel=df2["Fuel_Type"]=="fines"
stand=df2["Stand_Type"]=="open"
df2=df2[location&fuel&stand]
df3=df2.groupby(pd.to_datetime(df2["Date_Collected"])).mean()
df3 = df3.reset_index()
df3["Date_Collected"]=df3["Date_Collected"].apply(lambda x:x.strftime('%Y-%m-%d'))
a=pd.to_datetime(df["TIMESTAMP"])
a.apply(lambda x:x.strftime('%Y-%m-%d'))
df["TIMESTAMP"]=a
df["TIMESTAMP"]=df["TIMESTAMP"].apply(lambda x:x.strftime('%Y-%m-%d'))
final=pd.merge(left=df,right=df3,left_on="TIMESTAMP",right_on="Date_Collected")
#calculate the difference
final["Difference"]=final["FMC"]-final["Moisture_Percent"]
plt.plot(final["FMC"],final["Moisture_Percent"],"o")
#add a 1:1 line in the plot
x=np.linspace(0,170)
y=x
plt.plot(x,y,'k-')
plt.xlabel("estimate FMC(%)")
plt.ylabel("observation FMC(%)")
plt.title("Fine Fuel MC results comparision at NF, open, 12 couples")

#calculate statistic results out 
MSE = np.square(np.subtract(final["Moisture_Percent"],final["FMC"])).mean() 
 
RMSE = round(math.sqrt(MSE),2)
print("RMSE = ",RMSE)

#using IQR method to do exclusive the outlayers 
Q1 = final["Difference"].quantile(0.25)
Q3= final["Difference"].quantile(0.75)
IQR = Q3 - Q1

final= final[~((final["Difference"] < (Q1 - 1.5 * IQR)) |(final["Difference"] > (Q3 + 1.5 * IQR)))]
plt.plot(final["FMC"],final["Moisture_Percent"],"o")
plt.xlabel("estimate FMC(%)")
plt.ylabel("observation FMC(%)")
plt.title("Fine Fuel MC results comparision at NF, open")
x=np.linspace(0,45)
y=x
plt.plot(x,y,'k-')
MSE = np.square(np.subtract(final["Moisture_Percent"],final["FMC"])).mean() 
 
RMSE = round(math.sqrt(MSE),2)
print("Root Mean Square Error:\n")
print(RMSE)



In [ ]:
# week 4(24th-31st)

#weather condition> DMC>FMC
from math import e
DMCPrev=10 #assume numebr as a start 
count=0#data frame length 
Temp=df["AirTC"]
RH=df["RH"]
RAIN=df["Pg_ar_Rain"]
dayOfYear=df["dayOfYear"]
DMC_List=[]
while count<108:
    DMCPrev=DMC(Temp[count],RH[count],RAIN[count],DMCPrev,dayOfYear[count], 54)
    #what is appened downbelow is actuall FMC
    DMC_List.append(20+e**((DMCPrev-244.72)/(-43.43)))
    count+=1
df["Dmc"]=DMC_List
print (df)

#combining of field data +emperical data for Duff
fields2=["Location","Stand_Type","Fuel_Type","Date_Collected","Moisture_Percent"]
df2=pd.read_csv("2021PgFuel.csv",skipinitialspace=True,usecols=fields2)
location=df2["Location"]=="NFraser"
fuel=df2["Fuel_Type"]=="duff"
stand=df2["Stand_Type"]=="open"
df2=df2[location&fuel&stand]
df3=df2.groupby(pd.to_datetime(df2["Date_Collected"])).mean()
df3 = df3.reset_index()
df3["Date_Collected"]=df3["Date_Collected"].apply(lambda x:x.strftime('%Y-%m-%d'))
a=pd.to_datetime(df["TIMESTAMP"])
a.apply(lambda x:x.strftime('%Y-%m-%d'))
df["TIMESTAMP"]=a
df["TIMESTAMP"]=df["TIMESTAMP"].apply(lambda x:x.strftime('%Y-%m-%d'))
final=pd.merge(left=df,right=df3,left_on="TIMESTAMP",right_on="Date_Collected")
#apply difference to compare
final["Difference"]=final["Dmc"]-final["Moisture_Percent"]
plt.plot(final["Dmc"],final["Moisture_Percent"],"o")
x=np.linspace(0,300)
y=x
plt.plot(x,y,'k-')
plt.xlabel("estimate FMC(%)")
plt.ylabel("observation FMC(%)")
plt.title("Duff MC results comparision at NF, open")

MSE = np.square(np.subtract(final["Moisture_Percent"],final["Dmc"])).mean() 

RMSE = round(math.sqrt(MSE),2)
print("Root Mean Square Error:\n")
print(RMSE)

# using IQR to take out outlayers
Q1 = final["Difference"].quantile(0.25)
Q3= final["Difference"].quantile(0.75)
IQR = Q3 - Q1

final= final[~((final["Difference"] < (Q1 - 1.5 * IQR)) |(final["Difference"] > (Q3 + 1.5 * IQR)))]
plt.plot(final["Dmc"],final["Moisture_Percent"],"o")
plt.xlabel("estimate FMC(%)")
plt.ylabel("observation FMC(%)")
plt.title("Duff MC results comparision at NF, open")
x=np.linspace(0,230)
y=x
plt.plot(x,y,'k-')
MSE = np.square(np.subtract(final["Moisture_Percent"],final["Dmc"])).mean() 
 
RMSE = round(math.sqrt(MSE),2)
print("Root Mean Square Error:\n")
print(RMSE)


#for mature and juv, open:
fields2=["Location","Stand_Type","Fuel_Type","Date_Collected","Moisture_Percent"]
df2=pd.read_csv("2021PgFuel.csv",skipinitialspace=True,usecols=fields2)
location=df2["Location"]=="NFraser"
fuel=df2["Fuel_Type"]=="duff"
stand1=df2["Stand_Type"]=="open"
stand2=df2["Stand_Type"]=="old"
stand3=df2["Stand_Type"]=="juvenile"
df_open=df2[location&fuel&stand1]
df_old=df2[location&fuel&stand2]
df_juv=df2[location&fuel&stand3]
#open time matching
df_open1=df_open.groupby(pd.to_datetime(df_open["Date_Collected"])).mean()
df_open1 = df_open1.reset_index()
df_open1["Date_Collected"]=df_open1["Date_Collected"].apply(lambda x:x.strftime('%Y-%m-%d'))
#old time matching
df_old1=df_old.groupby(pd.to_datetime(df_old["Date_Collected"])).mean()
df_old1 = df_old1.reset_index()
df_old1["Date_Collected"]=df_old1["Date_Collected"].apply(lambda x:x.strftime('%Y-%m-%d'))
#juv time matching
df_juv1=df_juv.groupby(pd.to_datetime(df_juv["Date_Collected"])).mean()
df_juv1 = df_juv1.reset_index()
df_juv1["Date_Collected"]=df_juv1["Date_Collected"].apply(lambda x:x.strftime('%Y-%m-%d'))
#time format matching
a=pd.to_datetime(df["TIMESTAMP"])
a.apply(lambda x:x.strftime('%Y-%m-%d'))
df["TIMESTAMP"]=a
df["TIMESTAMP"]=df["TIMESTAMP"].apply(lambda x:x.strftime('%Y-%m-%d'))
#merging process-final_open
finalopen=pd.merge(left=df,right=df_open1,left_on="TIMESTAMP",right_on="Date_Collected")
finalopen["Difference"]=finalopen["Dmc"]-finalopen["Moisture_Percent"]
plt.plot(finalopen["Dmc"],finalopen["Moisture_Percent"],"o",label="open standing")


MSE1 = np.square(np.subtract(finalopen["Moisture_Percent"],finalopen["Dmc"])).mean() 
 
RMSE1= round(math.sqrt(MSE1),2)
print("Open Root Mean Square Error:\n")
print(RMSE1)

#merging process-final_old
finalold=pd.merge(left=df,right=df_old1,left_on="TIMESTAMP",right_on="Date_Collected")
finalold["Difference"]=finalold["Dmc"]-finalold["Moisture_Percent"]
plt.plot(finalold["Dmc"],finalold["Moisture_Percent"],"o",label="old standing")


MSE2 = np.square(np.subtract(finalold["Moisture_Percent"],finalold["Dmc"])).mean() 
 
RMSE2 = round(math.sqrt(MSE2),2)
print("Old Root Mean Square Error:\n")
print(RMSE2)

#merging process-final_juv

finaljuv=pd.merge(left=df,right=df_juv1,left_on="TIMESTAMP",right_on="Date_Collected")
finaljuv["Difference"]=finaljuv["Dmc"]-finaljuv["Moisture_Percent"]
plt.plot(finaljuv["Dmc"],finaljuv["Moisture_Percent"],"o",label="juv standing")

MSE3 = np.square(np.subtract(finaljuv["Moisture_Percent"],finaljuv["Dmc"])).mean() 
RMSE3 = round(math.sqrt(MSE3),2)
print("Juv Root Mean Square Error:\n")
print(RMSE3)
x=np.linspace(0,300)
y=x
plt.plot(x,y,'k-')
plt.xlabel("estimate FMC(%)")
plt.ylabel("observation FMC(%)")
plt.title("Duff MC results comparision at NF")
plt.legend(loc="upper left")



In [ ]:
# week 5(Feb1-Feb7th)
#for mature and juv, open-FFMC, comparing:

fields2=["Location","Stand_Type","Fuel_Type","Date_Collected","Moisture_Percent"]
df2=pd.read_csv("2021PgFuel.csv",skipinitialspace=True,usecols=fields2)
location=df2["Location"]=="NFraser"
fuel=df2["Fuel_Type"]=="fines"
stand1=df2["Stand_Type"]=="open"
stand2=df2["Stand_Type"]=="old"
stand3=df2["Stand_Type"]=="juvenile"
df_open=df2[location&fuel&stand1]
df_old=df2[location&fuel&stand2]
df_juv=df2[location&fuel&stand3]
#open time matching
df_open1=df_open.groupby(pd.to_datetime(df_open["Date_Collected"])).mean()
df_open1 = df_open1.reset_index()
df_open1["Date_Collected"]=df_open1["Date_Collected"].apply(lambda x:x.strftime('%Y-%m-%d'))
#old time matching
df_old1=df_old.groupby(pd.to_datetime(df_old["Date_Collected"])).mean()
df_old1 = df_old1.reset_index()
df_old1["Date_Collected"]=df_old1["Date_Collected"].apply(lambda x:x.strftime('%Y-%m-%d'))
#juv time matching
df_juv1=df_juv.groupby(pd.to_datetime(df_juv["Date_Collected"])).mean()
df_juv1 = df_juv1.reset_index()
df_juv1["Date_Collected"]=df_juv1["Date_Collected"].apply(lambda x:x.strftime('%Y-%m-%d'))
#time format matching
a=pd.to_datetime(df["TIMESTAMP"])
a.apply(lambda x:x.strftime('%Y-%m-%d'))
df["TIMESTAMP"]=a
df["TIMESTAMP"]=df["TIMESTAMP"].apply(lambda x:x.strftime('%Y-%m-%d'))
#merging process-final_open
finalopen=pd.merge(left=df,right=df_open1,left_on="TIMESTAMP",right_on="Date_Collected")
finalopen["Difference"]=finalopen["FMC"]-finalopen["Moisture_Percent"]
plt.plot(finalopen["FMC"],finalopen["Moisture_Percent"],"o",label="open standing")


MSE1 = np.square(np.subtract(finalopen["Moisture_Percent"],finalopen["FMC"])).mean() 
 
RMSE1= round(math.sqrt(MSE1),2)
print("Open Root Mean Square Error:\n")
print(RMSE1)

#merging process-final_old
finalold=pd.merge(left=df,right=df_old1,left_on="TIMESTAMP",right_on="Date_Collected")
finalold["Difference"]=finalold["FMC"]-finalold["Moisture_Percent"]
plt.plot(finalold["FMC"],finalold["Moisture_Percent"],"o",label="old standing")


MSE2 = np.square(np.subtract(finalold["Moisture_Percent"],finalold["FMC"])).mean() 
 
RMSE2 = round(math.sqrt(MSE2),2)
print("Old Root Mean Square Error:\n")
print(RMSE2)

#merging process-final_juv


finaljuv=pd.merge(left=df,right=df_juv1,left_on="TIMESTAMP",right_on="Date_Collected")
finaljuv["Difference"]=finaljuv["FMC"]-finaljuv["Moisture_Percent"]
plt.plot(finaljuv["FMC"],finaljuv["Moisture_Percent"],"o",label="juv standing")

MSE3 = np.square(np.subtract(finaljuv["Moisture_Percent"],finaljuv["FMC"])).mean() 
RMSE3 = round(math.sqrt(MSE3),2)
print("Juv Root Mean Square Error:\n")
print(RMSE3)
x=np.linspace(0,300)
y=x
plt.plot(x,y,'k-')
plt.xlabel("estimate FMC(%)")
plt.ylabel("observation FMC(%)")
plt.title("Fine fuel MC results comparision at NF")
plt.legend(loc="upper left")


#IQR process for duff
#open
Q1_open = finalopen["Difference"].quantile(0.25)
Q3_open= finalopen["Difference"].quantile(0.75)
IQR_open = Q3_open - Q1_open 

finalopen1= finalopen[~((finalopen["Difference"] < (Q1_open - 1.5 * IQR_open)) |(finalopen["Difference"] > (Q3_open + 1.5 * IQR_open)))]

plt.plot(finalopen1["Dmc"],finalopen1["Moisture_Percent"],"o",label="open standing")

MSE1 = np.square(np.subtract(finalopen1["Moisture_Percent"],finalopen1["Dmc"])).mean() 
 
RMSE1= round(math.sqrt(MSE1),2)
print("Open Root Mean Square Error:\n")
print(RMSE1)

#old
Q1_old = finalold["Difference"].quantile(0.25)
Q3_old= finalold["Difference"].quantile(0.75)
IQR_old = Q3_old - Q1_old 

finalold2= finalold[~((finalold["Difference"] < (Q1_old - 1.5 * IQR_old)) |(finalold["Difference"] > (Q3_old + 1.5 * IQR_old)))]

plt.plot(finalold2["Dmc"],finalold2["Moisture_Percent"],"o",label="old standing")


MSE2 = np.square(np.subtract(finalold2["Moisture_Percent"],finalold2["Dmc"])).mean() 
 
RMSE2= round(math.sqrt(MSE2),2)
print("Old Root Mean Square Error:\n")
print(RMSE2)

#juv
Q1_juv = finaljuv["Difference"].quantile(0.25)
Q3_juv= finaljuv["Difference"].quantile(0.75)
IQR_juv = Q3_juv - Q1_juv 

finaljuv2= finaljuv[~((finaljuv["Difference"] < (Q1_juv - 1.5 * IQR_juv)) |(finaljuv["Difference"] > (Q3_juv + 1.5 * IQR_juv)))]

plt.plot(finaljuv2["Dmc"],finaljuv2["Moisture_Percent"],"o",label="juv standing")


MSE3 = np.square(np.subtract(finaljuv2["Moisture_Percent"],finaljuv2["Dmc"])).mean() 
 
RMSE3= round(math.sqrt(MSE3),2)
print("Old Root Mean Square Error:\n")
print(RMSE3)
x=np.linspace(0,300)
y=x
plt.plot(x,y,'k-')
plt.xlabel("estimate FMC(%)")
plt.ylabel("observation FMC(%)")
plt.title("Duff MC results comparision at NF after IQR")
plt.legend(loc="upper left")




#IQR process for open
#open
Q1_open = finalopen["Difference"].quantile(0.25)
Q3_open= finalopen["Difference"].quantile(0.75)
IQR_open = Q3_open - Q1_open 

finalopen1= finalopen[~((finalopen["Difference"] < (Q1_open - 1.5 * IQR_open)) |(finalopen["Difference"] > (Q3_open + 1.5 * IQR_open)))]

plt.plot(finalopen1["FMC"],finalopen1["Moisture_Percent"],"o",label="open standing")

MSE1 = np.square(np.subtract(finalopen1["Moisture_Percent"],finalopen1["FMC"])).mean() 
 
RMSE1= round(math.sqrt(MSE1),2)
print("Open Root Mean Square Error:\n")
print(RMSE1)

#old
Q1_old = finalold["Difference"].quantile(0.25)
Q3_old= finalold["Difference"].quantile(0.75)
IQR_old = Q3_old - Q1_old 

finalold2= finalold[~((finalold["Difference"] < (Q1_old - 1.5 * IQR_old)) |(finalold["Difference"] > (Q3_old + 1.5 * IQR_old)))]

plt.plot(finalold2["FMC"],finalold2["Moisture_Percent"],"o",label="old standing")


MSE2 = np.square(np.subtract(finalold2["Moisture_Percent"],finalold2["FMC"])).mean() 
 
RMSE2= round(math.sqrt(MSE2),2)
print("Old Root Mean Square Error:\n")
print(RMSE2)

#juv
Q1_juv = finaljuv["Difference"].quantile(0.25)
Q3_juv= finaljuv["Difference"].quantile(0.75)
IQR_juv = Q3_juv - Q1_juv 

finaljuv2= finaljuv[~((finaljuv["Difference"] < (Q1_juv - 1.5 * IQR_juv)) |(finaljuv["Difference"] > (Q3_juv + 1.5 * IQR_juv)))]

plt.plot(finaljuv2["FMC"],finaljuv2["Moisture_Percent"],"o",label="juv standing")


MSE3 = np.square(np.subtract(finaljuv2["Moisture_Percent"],finaljuv2["FMC"])).mean() 
 
RMSE3= round(math.sqrt(MSE3),2)
print("Old Root Mean Square Error:\n")
print(RMSE3)
x=np.linspace(0,300)
y=x
plt.plot(x,y,'k-')
plt.xlabel("estimate FMC(%)")
plt.ylabel("observation FMC(%)")
plt.title("fine fuel MC results comparision at NF after IQR")
plt.legend(loc="upper left")

In [ ]:
# week 5(Feb1-Feb7th)-Remote sensing part, lowess analysis applied for estimate the satelite data:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.dates as md
import numpy as np
import time
import datetime
import pylab 
import statsmodels.api as sm
import seaborn as sns             
from matplotlib.dates import HourLocator, DayLocator, DateFormatter
from statsmodels.nonparametric.smoothers_lowess import lowess
#read files
df=pd.read_csv("2021PgFuel.csv")
df1=pd.read_csv("Bed_30.csv")
#seprate different stand types 
a1=df['Stand_Type']=="old"
a2=df['Stand_Type']=="open"
a3=df['Stand_Type']=="juvenile"
b1=df["Location"]=="Bednesti"
c1=df['Fuel_Type']=="duff"
c2=df["Fuel_Type"]=="fines"
c3=df["Fuel_Type"]=="foliage"
#seprate the observation data for foliage
#old
df2=df[a1&b1&c3]
#open
df3=df[a2&b1&c3]
#juv
df4=df[a3&b1&c3]
#seprate the ndmi
#old
d1=df1["featureID"]=="old"
#open
d2=df1["featureID"]=="open"
#juv
d3=df1["featureID"]=="juv"

#seprate data frame 
#old-ndmi
df5=df1[d1]
#open-ndmi
df6=df1[d2]
#juv-ndmi
df7=df1[d3]

#lowess analyze for old site 
#transfer datatime to days for observation data
sr=df2["Date_Collected"]
sr=pd.to_datetime(sr)
result=sr.dt.dayofyear
df2["days"]=result
#lowess for observation data 
y_hat1 = lowess(df2["Moisture_Percent"], df2["days"]) # note, default frac=2/3
y_hat2 = lowess(df2["Moisture_Percent"], df2["days"], frac=1/5)#more real


##transfer datatime to days for ndmi data
sr1=df5["Date"]
sr1=pd.to_datetime(sr1)
result1=sr1.dt.dayofyear
df5["days"]=result1
##lowess for ndmi data 
y_hat3 = lowess(df5["first"], df5["days"]) # note, default frac=2/3
y_hat4 = lowess(df5["first"], df5["days"], frac=1/5)#more real

#to plot for y_hat1 from observation lowess 
A1=[]
for i1 in range(len(y_hat1)):
    A1.append(y_hat1[i1][0])
B1=[]
for j1 in range(len(y_hat1)):
    B1.append(y_hat1[j1][1])
#extract the values in plot
C1=[]
C1_e=int(max(A1))
C1_s=int(min(A1))
C11=[]
for t1 in range(C1_s,C1_e):
    C1.append(round(np.interp(t1, A1,B1),3))
    C11.append(t1)
#convert them to df
AA1=pd.DataFrame()
AA1["mc_lowess"]=C1
AA1["days_lowess"]=C11
print(AA1)
#to plot for y_hat3 from ndmi lowess 
A3=[]
for i3 in range(len(y_hat3)):
    A3.append(y_hat3[i3][0])
B3=[]
for j3 in range(len(y_hat3)):
    B3.append(y_hat3[j3][1])
#extract the values in plot
C3=[]
C3_e=int(max(A3))
C3_s=int(min(A3))
C33=[]
for t3 in range(C3_s,C3_e):
    C3.append(round(np.interp(t3, A3,B3),3))
    C33.append(t3)
#convert them to df
AA3=pd.DataFrame()
AA3["ndmi_lowess"]=C3
AA3["days_lowess"]=C33
print(AA3)
#merge observation and ndmi-lowess-smooth default 
final_pair1=pd.merge(left=AA1,right=AA3,left_on="days_lowess",right_on="days_lowess")
print(final_pair1)
plt.plot(final_pair1["mc_lowess"],final_pair1["ndmi_lowess"],"o")

#to plot for a more real observation data
A2=[]
for i2 in range(len(y_hat2)):
    A2.append(y_hat2[i2][0])
B2=[]
for j2 in range(len(y_hat2)):
    B2.append(y_hat2[j2][1])
    
#to plot for a more real ndmi data 
A4=[]
for i4 in range(len(y_hat4)):
    A4.append(y_hat4[i4][0])
B4=[]
for j4 in range(len(y_hat4)):
    B4.append(y_hat4[j4][1])
#extract the values in plot for observation
C2=[]
C2_e=int(max(A2))
C2_s=int(min(A2))
C22=[]
for t2 in range(C2_s,C2_e):
    C2.append(round(np.interp(t2, A2,B2),3))
    C22.append(t2)
#convert them to df
AA2=pd.DataFrame()
AA2["mc_lowess"]=C2
AA2["days_lowess"]=C22
print(AA2)
#extract the values in plot for ndmi
C4=[]
C4_e=int(max(A4))
C4_s=int(min(A4))
C44=[]
for t4 in range(C4_s,C4_e):
    C4.append(round(np.interp(t4, A4,B4),3))
    C44.append(t4)
#convert them to df
AA4=pd.DataFrame()
AA4["ndmi_lowess"]=C4
AA4["days_lowess"]=C44
print(AA4)

#merge observation and ndmi-lowess-smooth more real 
final_pair2=pd.merge(left=AA2,right=AA4,left_on="days_lowess",right_on="days_lowess")
print(final_pair2)
plt.plot(final_pair2["mc_lowess"],final_pair2["ndmi_lowess"],"o")

In [ ]:
##Week 7(java, Feb 15-17), this is writen in java in google earth engine, it has different indices collection model, cloud filter, try it out! 
//write RS indices functions
//NDVI
function getNDVI(img) {
 
  var b1 = img.select('B8');
  var b2 = img.select('B4');
 
  var image_ndvi = b1.subtract(b2).divide(b1.add(b2)).rename('NDVI');
 
  return img.addBands(image_ndvi)

}
//NDMI
function getNDMI(img) {
 
  var b1 = img.select('B8A');
  var b2 = img.select('B11');
 
  var image_ndmi = b1.subtract(b2).divide(b1.add(b2)).rename('NDMI');
 
  return img.addBands(image_ndmi)

}

//NDWI
function getNDWI(img) {
 
  var b1 = img.select('B3');
  var b2 = img.select('B8');
 
  var image_ndwi = b1.subtract(b2).divide(b1.add(b2)).rename('NDWI');
 
  return img.addBands(image_ndwi)

}

//NMDI
function getNMDI(img) {
 
  var b1 = img.select('B8A');
  var b2 = img.select('B11');
  var b3 = img.select('B12');
 
  var image_nmdi = b1.subtract(b2).add(b3).divide(b1.add(b2).subtract(b3)).rename('NMDI');
 
  return img.addBands(image_nmdi)

}

//VARI
function getVARI(img) {
 
  var b1 = img.select('B3');
  var b2 = img.select('B4');
  var b3= img.select('B2');
 
  var image_vari = b1.subtract(b2).divide(b1.subtract(b2).subtract(b3)).rename('VARI');
 
  return img.addBands(image_vari)
}
var getQABits = function(image, start, end, newName) {
    // Compute the bits we need to extract.
    var pattern = 0;
    for (var i = start; i <= end; i++) {
       pattern += Math.pow(2, i);
    }
    // Return a single band image of the extracted QA bits, giving the band
    // a new name.
    return image.select([0], [newName])
                  .bitwiseAnd(pattern)
                  .rightShift(start);
};

var getshadowbits=function(image,newName){
  return image.select([0],[newName])
}
// A function to mask out cloudy pixels.
var clouds = function(image) {
  // Select the QA band.
  var QA = image.select(['QA60']);
  // Get the internal_cloud_algorithm_flag bit.
  return getQABits(QA, 10,10, 'cloud');
  // Return an image masking out cloudy areas.
};

//a function to mask out cirrus clody pixels 
var cloud_cirrus = function(image) {
  // Select the QA band.
  var QA = image.select(['QA60']);
  // Get the internal_cloud_algorithm_flag bit.
  return getQABits(QA, 11,11, 'cloud_cirrus');
  // Return an image masking out cloudy areas.
};
// Define a different standing object at Bednesti,SC,NF,HS,CP,MCD



var open= [
  ee.Feature(ee.Geometry.Point(-123.37044,53.871334), {name: 'BED',stand:"open"}).buffer({'distance': 35
}),
  ee.Feature(ee.Geometry.Point(-122.53790840362969,53.61666478088408), {name: 'SC',stand:"open"}).buffer({'distance': 35
}),
  ee.Feature(ee.Geometry.Point(-122.4906471114409,54.24422366683115), {name: 'NF',stand:"open"}).buffer({'distance': 35
}),
  ee.Feature(ee.Geometry.Point(-126.61521957203071,54.510560458595215), {name: 'HS',stand:"open"}).buffer({'distance': 35
}),
  ee.Feature(ee.Geometry.Point(-126.62338922091105,54.88414223445725), {name: 'CP',stand:"open"}).buffer({'distance': 35
}),
  ee.Feature(ee.Geometry.Point(-127.51496699805071,54.78263617031289), {name: 'MCD',stand:"open"}).buffer({'distance': 35
}),
];



//assign locations and points 
//JUV
var juvbed=ee.Geometry.Point(-123.37023,53.8701).buffer({'distance': 35
});
var juvsc=ee.Geometry.Point(-122.53568,53.6161).buffer({'distance': 35
});
var juvnf=ee.Geometry.Point(-122.48724,54.2440).buffer({'distance': 35
});
var juvhs=ee.Geometry.Point(-126.6145329265229,54.50762042679731).buffer({'distance': 35
});
var juvcp=ee.Geometry.Point(-126.6371221310673,54.88303133721306).buffer({'distance': 35
});
var juvmcd=ee.Geometry.Point(-127.52016195737143,54.78083388146724).buffer({'distance': 35
})
var juv_bed = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: juvbed,
  scale: 30
});
var juv_sc = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: juvsc,
  scale: 30
});
var juv_nf = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: juvnf,
  scale: 30
});
var juv_hs = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: juvhs,
  scale: 30
});
var juv_cp = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: juvcp,
  scale: 30
});
var juv_mcd = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: juvmcd,
  scale: 30
});

//Make gridded points per each lat and long for juv
var juv_bed = ee.FeatureCollection(juv_bed.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "juv",
        'loc':'BED'
      });
    });
var juv_sc = ee.FeatureCollection(juv_sc.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "juv",
        'loc':'SC'
      });
    });
var juv_nf = ee.FeatureCollection(juv_nf.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "juv",
        'loc':'NF'
      });
    });
var juv_hs = ee.FeatureCollection(juv_hs.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "juv",
        'loc':'HS'
      });
    });
    
var juv_cp = ee.FeatureCollection(juv_mcd.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "juv",
        'loc':'CP'
      });
    });
    
var juv_mcd = ee.FeatureCollection(juv_mcd.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "juv",
        'loc':'MCD'
      });
    });
var juv_points=juv_bed.merge(juv_sc).merge(juv_nf).merge(juv_hs).merge(juv_cp).merge(juv_mcd);

//old
var oldbed=ee.Geometry.Point(-123.36881, 53.872643).buffer({'distance': 35
});
var oldsc=  ee.Geometry.Point(-122.53694697444689,53.614360884145256).buffer({'distance': 35
});
var oldnf=  ee.Geometry.Point(-122.48998222029618,54.24231946275022).buffer({'distance': 35
});
var oldhs=  ee.Geometry.Point(-126.61225841327827,54.512005481417084).buffer({'distance': 35
});
var oldcp= ee.Geometry.Point(-126.64212176867106,54.88426566559375).buffer({'distance': 35
});
var oldmcd=ee.Geometry.Point(-127.5061049794655,54.78168334500671).buffer({'distance': 35
});


var old_bed = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: oldbed,
  scale: 30
});
var old_sc = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: oldsc,
  scale: 30
});
var old_nf = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: oldnf,
  scale: 30
});
var old_hs = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: oldhs,
  scale: 30
});
var old_cp = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: oldcp,
  scale: 30
});
var old_mcd = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: oldmcd,
  scale: 30
});

//Make gridded points per each lat and long for juv
var old_bed = ee.FeatureCollection(old_bed.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "old",
        'loc':'BED'
      });
    });
var old_sc = ee.FeatureCollection(old_sc.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "old",
        'loc':'SC'
      });
    });
var old_nf = ee.FeatureCollection(old_nf.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "old",
        'loc':'NF'
      });
    });
var old_hs = ee.FeatureCollection(old_hs.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "old",
        'loc':'HS'
      });
    });
    
var old_cp = ee.FeatureCollection(old_mcd.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "old",
        'loc':'CP'
      });
    });
    
var old_mcd = ee.FeatureCollection(old_mcd.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "old",
        'loc':'MCD'
      });
    });
    
var old_points=old_bed.merge(old_sc).merge(old_nf).merge(old_hs).merge(old_cp).merge(old_mcd);    
var openbed= 
  ee.Geometry.Point(-123.37044,53.871334).buffer({'distance': 35
});
var opensc= ee.Geometry.Point(-122.53790840362969,53.61666478088408).buffer({'distance': 35
});
var opennf=ee.Geometry.Point(-122.4906471114409,54.24422366683115).buffer({'distance': 35
});
var openhs=ee.Geometry.Point(-126.61521957203071,54.510560458595215).buffer({'distance': 35
});
var opencp= ee.Geometry.Point(-126.62338922091105,54.88414223445725).buffer({'distance': 35
});
var openmcd= ee.Geometry.Point(-127.51496699805071,54.78263617031289).buffer({'distance': 35
});




var open_bed = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: openbed,
  scale: 30
});
var open_sc = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: opensc,
  scale: 30
});
var open_nf = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: opennf,
  scale: 30
});
var open_hs = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: openhs,
  scale: 30
});
var open_cp = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: opencp,
  scale: 30
});
var open_mcd = ee.Image.pixelLonLat().reduceRegion({
  reducer: ee.Reducer.toCollection(['longitude', 'latitude']), 
  geometry: openmcd,
  scale: 30
});

//Make gridded points per each lat and long for juv
var open_bed = ee.FeatureCollection(open_bed.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "open",
        'loc':'BED'
      });
    });
var open_sc = ee.FeatureCollection(open_sc.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "open",
        'loc':'SC'
      });
    });
var open_nf = ee.FeatureCollection(open_nf.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "open",
        'loc':'NF'
      });
    });
var open_hs = ee.FeatureCollection(open_hs.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "open",
        'loc':'HS'
      });
    });
    
var open_cp = ee.FeatureCollection(open_mcd.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "open",
        'loc':'CP'
      });
    });
    
var open_mcd = ee.FeatureCollection(open_mcd.get('features'))
    .map(function(feature) {
      var lon = feature.get('longitude');
      var lat = feature.get('latitude');
      return ee.Feature(ee.Geometry.Point([lon, lat]), {
        'featureID': "open",
        'loc':'MCD'
      });
    });

var open_points=open_bed.merge(open_sc).merge(open_nf).merge(open_hs).merge(open_cp).merge(open_mcd);
    
var out_points = juv_points.merge(old_points).merge(open_points)
print(out_points, 'Final Sample Points')
Map.addLayer(out_points, null, 'Sample_Points');

var geo=[ee.Feature(geometry,{name: 'BED'}),
ee.Feature(geometry2,{name: 'SC'}),
ee.Feature(geometry3,{name: 'NF'}),
ee.Feature(geometry4,{name: 'HS'}),
ee.Feature(geometry5,{name: 'CP'}),
ee.Feature(geometry6,{name: 'MCD'})]

var geo = ee.FeatureCollection(geo)

//data
var S2 =ee.ImageCollection('COPERNICUS/S2')
          .filterDate('2021-06-01', '2021-09-30');

var c = S2.filterBounds(geo);
var c1 = c.map(function(image){return ee.Image(image).clip(geo)})

//first filter 
var withCloudiness = c1.map(function(image) {
  var cloud = clouds(image);
  var cloudiness = cloud.reduceRegion({
    reducer: 'mean', 
    geometry: geo, 
    scale: 30,
    crs:'EPSG:3857'
  });
  return image.set(cloudiness);
});

var withCloudiness1 = withCloudiness.map(function(image) {
  var cloud1 = cloud_cirrus(image);
  var cloudiness1 = cloud1.reduceRegion({
    reducer: 'mean', 
    geometry: geo, 
    scale: 30,
    crs:'EPSG:3857'
  });
  return image.set(cloudiness1);
});



var filteredCollection = withCloudiness1
            .filter(ee.Filter.and(
            ee.Filter.lt('cloud', 0.01),
            ee.Filter.lt('cloud_cirrus', 0.01)))
            .map(getNDVI);
var filteredCollection=filteredCollection.map(getNDMI);  
var filteredCollection=filteredCollection.map(getNDWI);
var filteredCollection=filteredCollection.map(getNMDI);
var filteredCollection=filteredCollection.map(getVARI);
print(filteredCollection)

//extract image values at points
//NDVI
var points_NDVI = filteredCollection.select('NDVI').map(function(image) {
  return image.reduceRegions({
    collection: out_points, 
    reducer: ee.Reducer.first(), //this is how the points are calculated 
    scale: 30,
  }).map(function(feature) {
    return feature.set({
      'imageID': image.id(),
      'Date':  ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
      'indices': "NDVI"
    })
  });
}).flatten();

//gt rid of nulls
points_NDVI = points_NDVI.filter(ee.Filter.notNull(['first']))
print(points_NDVI)
//NDMI
var points_NDMI = filteredCollection.select('NDMI').map(function(image) {
  return image.reduceRegions({
    collection: out_points, 
    reducer: ee.Reducer.first(), //this is how the points are calculated 
    scale: 30,
  }).map(function(feature) {
    return feature.set({
      'imageID': image.id(),
      'Date':  ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
      'indices': "NDMI"
    })
  });
}).flatten();

//gt rid of nulls
points_NDMI = points_NDMI.filter(ee.Filter.notNull(['first']))

//NDWI
var points_NDWI = filteredCollection.select('NDWI').map(function(image) {
  return image.reduceRegions({
    collection: out_points, 
    reducer: ee.Reducer.first(), //this is how the points are calculated 
    scale: 30,
  }).map(function(feature) {
    return feature.set({
      'imageID': image.id(),
      'Date':  ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
      'indices': "NDWI"
    })
  });
}).flatten();

//gt rid of nulls
points_NDWI = points_NDWI.filter(ee.Filter.notNull(['first']))

//NMDI
var points_NMDI = filteredCollection.select('NMDI').map(function(image) {
  return image.reduceRegions({
    collection: out_points, 
    reducer: ee.Reducer.first(), //this is how the points are calculated 
    scale: 30,
  }).map(function(feature) {
    return feature.set({
      'imageID': image.id(),
      'Date':  ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
      'indices': "NMDI"
    })
  });
}).flatten();

//gt rid of nulls
points_NMDI = points_NMDI.filter(ee.Filter.notNull(['first']))


//VARI
var points_VARI = filteredCollection.select('VARI').map(function(image) {
  return image.reduceRegions({
    collection: out_points, 
    reducer: ee.Reducer.first(), //this is how the points are calculated 
    scale: 30,
  }).map(function(feature) {
    return feature.set({
      'imageID': image.id(),
      'Date':  ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
      'indices': "VARI"
    })
  });
}).flatten();

//gt rid of nulls
points_VARI = points_VARI.filter(ee.Filter.notNull(['first']))
//check

//merge together 
var upper_a=points_NDVI.merge(points_NDMI).merge(points_NDWI).merge(points_NMDI).merge(points_VARI)

print(upper_a)
//export to drive
Export.table.toDrive({
  collection: upper_a, 
  description:"bednesti"+ 2021, 
  fileNamePrefix: 'Diff_stand_NDMI'+ 2021, 
  fileFormat: 'CSV',
  folder: 'GEOG650'
}); 

In [ ]:
#Week 8 Feb 28th-March5th
#min-max test, some statistic work 
#bed
dfbed=[]
dfbed=pd.DataFrame(dfbed)
dfbed=dfbed.append(Bed_R,ignore_index=True).append(Bed_R1, ignore_index=True)
dfbed["nor"] = ""
a=max(dfbed["ndmi_lowess"])
b=min(dfbed["ndmi_lowess"])
c=dfbed["ndmi_lowess"]
for i in range(len(c)):
    dfbed["nor"][i]=(c[i]-b)/(a-b)

    
#NF    
dfnf=[]
dfnf=pd.DataFrame(dfnf)
dfnf=dfnf.append(NF_R,ignore_index=True).append(NF_R1, ignore_index=True)
dfnf["nor"] = ""
a=max(dfnf["ndmi_lowess"])
b=min(dfnf["ndmi_lowess"])
c=dfnf["ndmi_lowess"]
for i in range(len(c)):
    dfnf["nor"][i]=(c[i]-b)/(a-b)
    
#sc
dfsc=[]
dfsc=pd.DataFrame(dfsc)
dfsc=dfsc.append(SC_R,ignore_index=True).append(SC_R1, ignore_index=True)
dfsc["nor"] = ""
a=max(dfsc["ndmi_lowess"])
b=min(dfsc["ndmi_lowess"])
c=dfsc["ndmi_lowess"]
for i in range(len(c)):
    dfsc["nor"][i]=(c[i]-b)/(a-b)

#HS   
dfhs=[]
dfhs=pd.DataFrame(dfbed)
dfhs=dfhs.append(HS_R,ignore_index=True).append(HS_R1, ignore_index=True)
dfhs["nor"] = ""
a=max(dfhs["ndmi_lowess"])
b=min(dfhs["ndmi_lowess"])
c=dfhs["ndmi_lowess"]
for i in range(len(c)):
    dfhs["nor"][i]=(c[i]-b)/(a-b)


#CP
dfcp=[]
dfcp=pd.DataFrame(dfcp)
dfcp=dfcp.append(CP_R,ignore_index=True).append(CP_R1, ignore_index=True)
dfcp["nor"] = ""
a=max(dfcp["ndmi_lowess"])
b=min(dfcp["ndmi_lowess"])
c=dfcp["ndmi_lowess"]
for i in range(len(c)):
    dfcp["nor"][i]=(c[i]-b)/(a-b)
    
#MCD
dfmcd=[]
dfmcd=pd.DataFrame(dfmcd)
dfmcd=dfmcd.append(MCD_R,ignore_index=True).append(MCD_R1, ignore_index=True)
dfmcd["nor"] = ""
a=max(dfmcd["ndmi_lowess"])
b=min(dfmcd["ndmi_lowess"])
c=dfmcd["ndmi_lowess"]
for i in range(len(c)):
    dfmcd["nor"][i]=(c[i]-b)/(a-b)
    

dfNDMI=[]
dfNDMI=pd.DataFrame(dfNDMI)
dfNDMI=dfNDMI.append(dfbed, ignore_index=True).append(dfnf, ignore_index=True).append(dfsc, ignore_index=True).append(dfhs, ignore_index=True).append(dfcp, ignore_index=True).append(dfmcd, ignore_index=True)

import scipy.stats
plt.plot(dfNDMI["nor"],dfNDMI["Moisture_Percent"],"o")
plt.xlabel('nor') 
plt.ylabel('MC') 
  

plt.title("juv MC vs Normalized NDMI")
m, b = np.polyfit(dfNDMI["nor"].astype(str).astype(float),dfNDMI["Moisture_Percent"], 1)
print("mc=",round(m,2),"nor+",round(b,2))

plt.plot(dfNDMI["nor"], m*dfNDMI["nor"] + b)

import math
#RMSE
Rr=[]
Rp=[]
for i in range(len(dfNDMI)):
    Rr.append(dfNDMI["Moisture_Percent"][i])
for j in range(len(dfNDMI)):
    Rp.append(26.53*dfNDMI["nor"]+100.27)
MSE = np.square(np.subtract(Rr,Rp)).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE) 
print("correlation value for NDMI", format(scipy.stats.pearsonr(dfNDMI["nor"], dfNDMI["Moisture_Percent"])[0],"2f"))
print("P value for NDMI", format(scipy.stats.pearsonr(dfNDMI["nor"], dfNDMI["Moisture_Percent"])[1],"2f"))



In [ ]:
#Week 9 March 6th to March 13th, functions build up, shorten my coding for remote sensing work, a lot of IQR filter fuctions and linear regression function. feel free to borrow:

#functions
def iqr_observ(df):
    df["days"]=pd.to_datetime(df["Date_Collected"]).dt.dayofyear
    a=df.groupby("days")['Moisture_Percent'].apply(list).reset_index(name='new')
    a.index.name = None
    b=[]
    c=[]
    for i in range(len(a)):
        q1 = np.quantile(a["new"][i],0.25)
        q3 = np.quantile(a["new"][i],0.75)
        iqr = q3-q1 #Interquartile range
        fence_low  = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        for j in range(len(a["new"][i])):
            if (a["new"][i][j] > fence_low) & (a["new"][i][j] < fence_high):
                b.append(a["new"][i][j])
                c.append(a["days"][i])
    d = {'days':c,'Moisture_Percent':b}
    df = pd.DataFrame(d) 
    return df

def iqr_indices(df):
    a=df.groupby("Date")['first'].apply(list).reset_index(name='new')
    a.index.name = None
    b=[]
    c=[]
    for i in range(len(a)):
        q1 = np.quantile(a["new"][i],0.25)
        q3 = np.quantile(a["new"][i],0.75)
        iqr = q3-q1 #Interquartile range
        fence_low  = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        for j in range(len(a["new"][i])):
            if (a["new"][i][j] > fence_low) & (a["new"][i][j] < fence_high):
                b.append(a["new"][i][j])
                c.append(a["Date"][i])
    d = {'Date':c,'first':b}
    df = pd.DataFrame(d)
    return df

#a is start date, b is end date c is fraction for lowess 
def Lowess(df, a, b, c):
    df["days"]=pd.to_datetime(df["Date"]).dt.dayofyear
    df= df[(df['days'] > a) & (df['days'] <= b)]

    y_hat3 = lowess(df["first"],df["days"],frac=c) 
    #merge process 
    A3=[]
    for i3 in range(len(y_hat3)):
        A3.append(y_hat3[i3][0])
    B3=[]
    for j3 in range(len(y_hat3)):
        B3.append(y_hat3[j3][1])
    #extract the values in plot
    
    C3=[]
    C3_e=int(max(A3))
    C3_s=int(min(A3))
    C33=[]
    for t3 in range(C3_s,C3_e):
        C3.append(round(np.interp(t3, A3,B3),3))
        C33.append(t3)
    #convert them to df

    Bed_NDMI_lowess_D1=pd.DataFrame()
    Bed_NDMI_lowess_D1["ndmi_lowess"]=C3
    Bed_NDMI_lowess_D1["days_lowess"]=C33
    return Bed_NDMI_lowess_D1

#df1 is the 
def merge(df1,df2):
    df1["ndmi_lowess"]=""
    for i in range(len(df1)):
        for j in range(len(df2)):
            if df1["days"][i] == df2["days_lowess"][j]:
                df1["ndmi_lowess"][i]=df2["ndmi_lowess"][j]
    return  df1
#data solution
def fix(df):
    nan_value = float("NaN")
    df.replace("", nan_value, inplace=True)
    df.dropna(subset = ["ndmi_lowess"], inplace=True)
    return df

#a is k, b is b
def iqr_final(df,a,b):
    
    Rr=[]
    Rp=[]

    for i in range(len(df)):
        Rr.append(df["Moisture_Percent"][i])
    for j in range(len(df)):
        Rp.append(a*df["ndmi_lowess"][j]+b)
    df["Rr"]=Rr
    df["Rp"]=Rp
    FF=df["Rr"]-df["Rp"]
    df["FF"]=abs(FF)
    Q1 = df["FF"].quantile(0.25)
    Q3 = df["FF"].quantile(0.75)
    IQR = Q3 - Q1
    df=df.query('(@Q1 - 1.5 * @IQR) <= FF<= (@Q3 + 1.5 * @IQR)')
    return df

In [ ]:
#week 10 March 14th-Marth 21st: linear regression from last week applied in post fire area, this is an example .
fig,ax=plt.subplots(2, figsize=(10,6),sharex=True)
L1=ax[0].plot(pd.to_datetime(juvVARI["Date"]),juvVARI["first"]*(142.96)+191.74,"o",color="b",label="juv-VARI")
L2=ax[0].plot(pd.to_datetime(matVARI["Date"]),matVARI["first"]*136.32+188.85,"o",color="r",label="mat-VARi")
L=L1+L2
labs=[l.get_label()for l in L]
ax[0].legend(L,labs,loc=2)
L11=ax[1].plot(pd.to_datetime(juvNDVI["Date"]),juvNDVI["first"]*(-124.74)+201.31,"o",color="b",label="juv-NDVI")
L22=ax[1].plot(pd.to_datetime(matNDVI["Date"]),matNDVI["first"]*(-268.35)+300.09,"o",color="r",label="mat-NDVI")
LL=L11+L22
labs1=[l.get_label()for l in LL]
ax[1].legend(LL,labs1,loc=2)
ax[0].set_ylabel("Moisture Percent")
ax[1].set_xlabel("Date")
ax[1].set_ylabel("Moisture Percent")
fig.suptitle("juv vs mat post fire MC")

In [ ]:
#week 11 Marth 22nd to Marth 29th: some java work in GEE(google earth engine), burn severity functions and coding!
//===========================================================================================
//             BURN SEVERITY MAPPING USING THE NORMALIZED BURN RATIO (NBR)
//===========================================================================================
// Normalized Burn Ratio will be applied to imagery from before and after a wild fire. By
// calculating the difference afterwards (dNBR) Burn Severity is derived, showing the spatial
// impact of the disturbance. Imagery used in this process comes from either Sentinel-2 or 
// Landsat 8.
//===========================================================================================

//:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
//                                    RUN A DEMO (optional)

// If you would like to run an example of mapping burn severity you can use the predefined 
// geometry below as well as the other predefined parameter settings. The code will take you
// to Empredado, Chile where wildfires devasted large forested areas in January and February 
// of 2017. 
// --> Remove the comment-symbol (//) below to so Earth Engine recognizes the polygon.

// Now hit Run to start the demo! 
// Do not forget to delete/outcomment this geometry before creating a new one!
//:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

//*******************************************************************************************
//                             SELECT YOUR OWN STUDY AREA   

// Use the polygon-tool in the top left corner of the map pane to draw the shape of your 
// study area. Single clicks add vertices, double-clicking completes the polygon.
// **CAREFUL**: Under 'Geometry Imports' (top left in map pane) uncheck the 
//                geometry box, so it does not block the view on the imagery later.

//*******************************************************************************************
//                                     SET TIME FRAME

// Set start and end dates of a period BEFORE the fire. Make sure it is long enough for 
// Sentinel-2 to acquire an image (repitition rate = 5 days). Adjust these parameters, if
// your ImageCollections (see Console) do not contain any elements.
var prefire_start = '2017-04-07';   
var prefire_end = '2017-07-07';

// Now set the same parameters for AFTER the fire.
var postfire_start = '2017-09-01';
var postfire_end = '2018-07-31';

//*******************************************************************************************
//                            SELECT A SATELLITE PLATFORM

// You can select remote sensing imagery from two availible satellite sensors. 
// Consider details of each mission below to choose the data suitable for your needs.

// Landsat 8                             |  Sentinel-2 (A&B)
//-------------------------------------------------------------------------------------------
// launched:        February 11th, 2015  |  June 23rd, 2015 & March 7th, 2017
// repitition rate: 16 days              |  5 day (since 2017)
// resolution:      30 meters            |  10 meters 
// advantages:      longer time series   |  9 times higher spatial detail
//                  smaller export file  |  higher chance of cloud-free images

// SELECT one of the following:   'L8'  or 'S2' 

var platform = 'L8';               // <--- assign your choice to the platform variable

//*******************************************************************************************
//---->>> DO NOT EDIT THE SCRIPT PAST THIS POINT! (unless you know what you are doing) <<<---
//------------------->>> NOW HIT 'RUN' AT THE TOP OF THE SCRIPT! <<<-------------------------
//--> THE FINAL BURN SEVERITY PRODUCT WILL READY FOR DOWNLOAD ON THE RIGHT (UNDER TASKS) <---

//*******************************************************************************************


//---------------------------------- Translating User Inputs --------------------------------

// Print Satellite platform and dates to console
if (platform == 'S2' | platform == 's2') {
  var ImCol = 'COPERNICUS/S2';
  var pl = 'Sentinel-2';
} else {
  var ImCol = 'LANDSAT/LC08/C01/T1_SR';
  var pl = 'Landsat 8';
}
print(ee.String('Data selected for analysis: ').cat(pl));
print(ee.String('Fire incident occurred between ').cat(prefire_end).cat(' and ').cat(postfire_start));

// Location
var area = ee.FeatureCollection(geometry);

// Set study area as map center.
Map.centerObject(area);

//----------------------- Select Landsat imagery by time and location -----------------------

var imagery = ee.ImageCollection(ImCol);

// In the following lines imagery will be collected in an ImageCollection, depending on the
// location of our study area, a given time frame and the ratio of cloud cover.
var prefireImCol = ee.ImageCollection(imagery
    // Filter by dates.
    .filterDate(prefire_start, prefire_end)
    // Filter by location.
    .filterBounds(area)
    .sort('CLOUD_COVER'));
    
// Select all images that overlap with the study area from a given time frame 
// As a post-fire state we select the 25th of February 2017
var postfireImCol = ee.ImageCollection(imagery
    // Filter by dates.
    .filterDate(postfire_start, postfire_end)
    // Filter by location.
    .filterBounds(area)
    .sort('CLOUD_COVER'));

// Add the clipped images to the console on the right
print("Pre-fire Image Collection: ", prefireImCol); 
print("Post-fire Image Collection: ", postfireImCol);

//------------------------------- Apply a cloud and snow mask -------------------------------

// Function to mask clouds from the pixel quality band of Sentinel-2 SR data.
function maskS2sr(image) {
  // Bits 10 and 11 are clouds and cirrus, respectively.
  var cloudBitMask = ee.Number(2).pow(10).int();
  var cirrusBitMask = ee.Number(2).pow(11).int();
  // Get the pixel QA band.
  var qa = image.select('QA60');
  // All flags should be set to zero, indicating clear conditions.
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0)
      .and(qa.bitwiseAnd(cirrusBitMask).eq(0));
  // Return the masked image, scaled to TOA reflectance, without the QA bands.
  return image.updateMask(mask)
      .copyProperties(image, ["system:time_start"]);
}

// Function to mask clouds from the pixel quality band of Landsat 8 SR data.
function maskL8sr(image) {
  // Bits 3 and 5 are cloud shadow and cloud, respectively.
  var cloudShadowBitMask = 1 << 3;
  var cloudsBitMask = 1 << 5;
  var snowBitMask = 1 << 4;
  // Get the pixel QA band.
  var qa = image.select('pixel_qa');
  // All flags should be set to zero, indicating clear conditions.
  var mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
      .and(qa.bitwiseAnd(cloudsBitMask).eq(0))
      .and(qa.bitwiseAnd(snowBitMask).eq(0));
  // Return the masked image, scaled to TOA reflectance, without the QA bands.
  return image.updateMask(mask)
      .select("B[0-9]*")
      .copyProperties(image, ["system:time_start"]);
}

// Apply platform-specific cloud mask
if (platform == 'S2' | platform == 's2') {
  var prefire_CM_ImCol = prefireImCol.map(maskS2sr);
  var postfire_CM_ImCol = postfireImCol.map(maskS2sr);
} else {
  var prefire_CM_ImCol = prefireImCol.map(maskL8sr);
  var postfire_CM_ImCol = postfireImCol.map(maskL8sr);
}

//----------------------- Mosaic and clip images to study area -----------------------------

// This is especially important, if the collections created above contain more than one image
// (if it is only one, the mosaic() does not affect the imagery).

var pre_mos = prefireImCol.mosaic().clip(area);
var post_mos = postfireImCol.mosaic().clip(area);

var pre_cm_mos = prefire_CM_ImCol.mosaic().clip(area);
var post_cm_mos = postfire_CM_ImCol.mosaic().clip(area);

// Add the clipped images to the console on the right
print("Pre-fire True Color Image: ", pre_mos); 
print("Post-fire True Color Image: ", post_mos);

//------------------ Calculate NBR for pre- and post-fire images ---------------------------

// Apply platform-specific NBR = (NIR-SWIR2) / (NIR+SWIR2)
if (platform == 'S2' | platform == 's2') {
  var preNBR = pre_cm_mos.normalizedDifference(['B8', 'B12']);
  var postNBR = post_cm_mos.normalizedDifference(['B8', 'B12']);
} else {
  var preNBR = pre_cm_mos.normalizedDifference(['B5', 'B7']);
  var postNBR = post_cm_mos.normalizedDifference(['B5', 'B7']);
}


// Add the NBR images to the console on the right
//print("Pre-fire Normalized Burn Ratio: ", preNBR); 
//print("Post-fire Normalized Burn Ratio: ", postNBR);

//------------------ Calculate difference between pre- and post-fire images ----------------

// The result is called delta NBR or dNBR
var dNBR_unscaled = preNBR.subtract(postNBR);

// Scale product to USGS standards
var dNBR = dNBR_unscaled.multiply(1000);

// Add the difference image to the console on the right
print("Difference Normalized Burn Ratio: ", dNBR);

//==========================================================================================
//                                    ADD LAYERS TO MAP

// Add boundary.
Map.addLayer(area.draw({color: 'ffffff', strokeWidth: 5}), {},'Study Area');

//---------------------------------- True Color Imagery ------------------------------------

// Apply platform-specific visualization parameters for true color images
if (platform == 'S2' | platform == 's2') {
  var vis = {bands: ['B4', 'B3', 'B2'], max: 2000, gamma: 1.5};
} else {
  var vis = {bands: ['B4', 'B3', 'B2'], min: 0, max: 4000, gamma: 1.5};
}

// Add the true color images to the map.
Map.addLayer(pre_mos, vis,'Pre-fire image');
Map.addLayer(post_mos, vis,'Post-fire image');

// Add the true color images to the map.
Map.addLayer(pre_cm_mos, vis,'Pre-fire True Color Image - Clouds masked');
Map.addLayer(post_cm_mos, vis,'Post-fire True Color Image - Clouds masked');

//--------------------------- Burn Ratio Product - Greyscale -------------------------------

var grey = ['white', 'black'];

// Remove comment-symbols (//) below to display pre- and post-fire NBR seperately
//Map.addLayer(preNBR, {min: -1, max: 1, palette: grey}, 'Prefire Normalized Burn Ratio');
//Map.addLayer(postNBR, {min: -1, max: 1, palette: grey}, 'Postfire Normalized Burn Ratio');

Map.addLayer(dNBR, {min: -1000, max: 1000, palette: grey}, 'dNBR greyscale');

//------------------------- Burn Ratio Product - Classification ----------------------------

// Define an SLD style of discrete intervals to apply to the image.
var sld_intervals =
  '<RasterSymbolizer>' +
    '<ColorMap type="intervals" extended="false" >' +
      '<ColorMapEntry color="#ffffff" quantity="-500" label="-500"/>' +
      '<ColorMapEntry color="#7a8737" quantity="-250" label="-250" />' +
      '<ColorMapEntry color="#acbe4d" quantity="-100" label="-100" />' +
      '<ColorMapEntry color="#0ae042" quantity="100" label="100" />' +
      '<ColorMapEntry color="#fff70b" quantity="270" label="270" />' +
      '<ColorMapEntry color="#ffaf38" quantity="440" label="440" />' +
      '<ColorMapEntry color="#ff641b" quantity="660" label="660" />' +
      '<ColorMapEntry color="#a41fd6" quantity="2000" label="2000" />' +
    '</ColorMap>' +
  '</RasterSymbolizer>';

// Add the image to the map using both the color ramp and interval schemes.
Map.addLayer(dNBR.sldStyle(sld_intervals), {}, 'dNBR classified');

// Seperate result into 8 burn severity classes
var thresholds = ee.Image([-1000, -251, -101, 99, 269, 439, 659, 2000]);
var classified = dNBR.lt(thresholds).reduce('sum').toInt();

//==========================================================================================
//                              ADD BURNED AREA STATISTICS

// count number of pixels in entire layer
var allpix =  classified.updateMask(classified);  // mask the entire layer
var pixstats = allpix.reduceRegion({
  reducer: ee.Reducer.count(),               // count pixels in a single class
  geometry: area,
  scale: 30
  });
var allpixels = ee.Number(pixstats.get('sum')); // extract pixel count as a number


// create an empty list to store area values in
var arealist = [];

// create a function to derive extent of one burn severity class
// arguments are class number and class name
var areacount = function(cnr, name) {
 var singleMask =  classified.updateMask(classified.eq(cnr));  // mask a single class
 var stats = singleMask.reduceRegion({
  reducer: ee.Reducer.count(),               // count pixels in a single class
  geometry: area,
  scale: 30
  });
var pix =  ee.Number(stats.get('sum'));
var hect = pix.multiply(900).divide(10000);                // Landsat pixel = 30m x 30m --> 900 sqm
var perc = pix.divide(allpixels).multiply(10000).round().divide(100);   // get area percent by class and round to 2 decimals
arealist.push({Class: name, Pixels: pix, Hectares: hect, Percentage: perc});
};

// severity classes in different order
var names2 = ['NA', 'High Severity', 'Moderate-high Severity',
'Moderate-low Severity', 'Low Severity','Unburned', 'Enhanced Regrowth, Low', 'Enhanced Regrowth, High'];

// execute function for each class
for (var i = 0; i < 8; i++) {
  areacount(i, names2[i]);
  }

print('Burned Area by Severity Class', arealist, '--> click list objects for individual classes');

//==========================================================================================
//                                    ADD A LEGEND

// set position of panel
var legend = ui.Panel({
  style: {
    position: 'bottom-left',
    padding: '8px 15px'
  }});
 
// Create legend title
var legendTitle = ui.Label({
  value: 'dNBR Classes',
  style: {fontWeight: 'bold',
    fontSize: '18px',
    margin: '0 0 4px 0',
    padding: '0'
    }});
 
// Add the title to the panel
legend.add(legendTitle);
 
// Creates and styles 1 row of the legend.
var makeRow = function(color, name) {
 
      // Create the label that is actually the colored box.
      var colorBox = ui.Label({
        style: {
          backgroundColor: '#' + color,
          // Use padding to give the box height and width.
          padding: '8px',
          margin: '0 0 4px 0'
        }});
 
      // Create the label filled with the description text.
      var description = ui.Label({
        value: name,
        style: {margin: '0 0 4px 6px'}
      });
 
      // return the panel
      return ui.Panel({
        widgets: [colorBox, description],
        layout: ui.Panel.Layout.Flow('horizontal')
      })};
 
//  Palette with the colors
var palette =['7a8737', 'acbe4d', '0ae042', 'fff70b', 'ffaf38', 'ff641b', 'a41fd6', 'ffffff'];
 
// name of the legend
var names = ['Enhanced Regrowth, High','Enhanced Regrowth, Low','Unburned', 'Low Severity',
'Moderate-low Severity', 'Moderate-high Severity', 'High Severity', 'NA'];
 
// Add color and and names
for (var i = 0; i < 8; i++) {
  legend.add(makeRow(palette[i], names[i]));
  }  
 
// add legend to map (alternatively you can also print the legend to the console)
Map.add(legend);

//==========================================================================================
//                                  PREPARE FILE EXPORT

var id = dNBR.id().getInfo();
